In [ ]:
# !pip install oracledb

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


In [ ]:
import oracledb as db
from sqlalchemy import create_engine, Connection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

from datetime import timedelta, date
from dateutil.relativedelta import relativedelta
from time import sleep

from getpass import getpass

# #Paralelização na veia
# from joblib import Parallel, delayed

# Configs
dsn = 'BIFC'
user=input('Informe o nome de usuário do SQL: ')
password= getpass('senha usuario: ')

inicio_ano = date(2023, 1, 1) #date(2025, 10, 1)    # ATT AJUSTAR DEPOIS PROS OUTROS ANOS
periods = (date.today().year - inicio_ano.year) * 12 + (date.today().month - inicio_ano.month) # relativedelta(date.today(), inicio_ano).months 
range_date = pd.date_range(start = inicio_ano, periods= periods, freq = 'MS')

In [88]:
inicio_ano = date(2023, 1, 1) #date(2025, 10, 1)    # ATT AJUSTAR DEPOIS PROS OUTROS ANOS
periods = (date.today().year - inicio_ano.year) * 12 + (date.today().month - inicio_ano.month) # relativedelta(date.today(), inicio_ano).months 
range_date = pd.date_range(start = inicio_ano, periods= periods, freq = 'MS')
print(f'Periodos: {periods}')
range_date = range_date.strftime('%Y%m')
print(f'Range Date: {range_date}')
range_date_start = pd.date_range(start = inicio_ano, periods= periods, freq = 'MS').strftime('%d-%m-%Y')
range_date_end = pd.date_range(start = inicio_ano, periods= periods, freq = 'ME').strftime('%d-%m-%Y')
print(f'Range date end: {range_date_end}')

Periodos: 36
Range Date: Index(['202301', '202302', '202303', '202304', '202305', '202306', '202307',
       '202308', '202309', '202310', '202311', '202312', '202401', '202402',
       '202403', '202404', '202405', '202406', '202407', '202408', '202409',
       '202410', '202411', '202412', '202501', '202502', '202503', '202504',
       '202505', '202506', '202507', '202508', '202509', '202510', '202511',
       '202512'],
      dtype='object')
Range date end: Index(['31-01-2023', '28-02-2023', '31-03-2023', '30-04-2023', '31-05-2023',
       '30-06-2023', '31-07-2023', '31-08-2023', '30-09-2023', '31-10-2023',
       '30-11-2023', '31-12-2023', '31-01-2024', '29-02-2024', '31-03-2024',
       '30-04-2024', '31-05-2024', '30-06-2024', '31-07-2024', '31-08-2024',
       '30-09-2024', '31-10-2024', '30-11-2024', '31-12-2024', '31-01-2025',
       '28-02-2025', '31-03-2025', '30-04-2025', '31-05-2025', '30-06-2025',
       '31-07-2025', '31-08-2025', '30-09-2025', '31-10-2025', '30-11-20

In [80]:
# Execução 1: Janeiro a Abril de 2025
# Execução 2: Maio a Agosto de 2025
# Execução 3: Setembro a Dezembro de 2025
# Execução 4: Setembro a Dezembro de 2024
# Execução 5: Maio a Agosto de 2024
# Execução 6: Janeiro a Abril de 2024
# Execução 7: Janeiro a Abril de 2023
# Execução 8: Maio a Agosto de 2023
# Execução 9: Setembro a Dezembro de 2023
# range_date_start = ['01-09-2023', '01-10-2023', '01-11-2023', '01-12-2023']
# range_date_end = ['30-09-2023', '31-10-2023', '30-11-2023', '31-12-2023']
# range_date_start = ['01-05-2025', '01-06-2025', '01-07-2025', '01-08-2025']
# range_date_end = ['31-05-2025', '30-06-2025', '31-07-2025', '31-08-2025']
range_date_start = ['01-01-2025', '01-02-2025', '01-03-2025', '01-04-2025']
range_date_end = ['31-01-2025', '28-02-2025', '31-03-2025', '30-04-2025']
periods = 4

In [4]:
periods

36

In [5]:
range_date = range_date.strftime('%Y%m')
range_date

Index(['202301', '202302', '202303', '202304', '202305', '202306', '202307',
       '202308', '202309', '202310', '202311', '202312', '202401', '202402',
       '202403', '202404', '202405', '202406', '202407', '202408', '202409',
       '202410', '202411', '202412', '202501', '202502', '202503', '202504',
       '202505', '202506', '202507', '202508', '202509', '202510', '202511',
       '202512'],
      dtype='object')

In [6]:
range_date_start = pd.date_range(start = inicio_ano, periods= periods, freq = 'MS').strftime('%d-%m-%Y')
range_date_end = pd.date_range(start = inicio_ano, periods= periods, freq = 'M').strftime('%d-%m-%Y')

C:\Users\mirella.farias\AppData\Local\Temp\ipykernel_24104\443390923.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  range_date_end = pd.date_range(start = inicio_ano, periods= periods, freq = 'M').strftime('%d-%m-%Y')


In [7]:
range_date_end

Index(['31-01-2023', '28-02-2023', '31-03-2023', '30-04-2023', '31-05-2023',
       '30-06-2023', '31-07-2023', '31-08-2023', '30-09-2023', '31-10-2023',
       '30-11-2023', '31-12-2023', '31-01-2024', '29-02-2024', '31-03-2024',
       '30-04-2024', '31-05-2024', '30-06-2024', '31-07-2024', '31-08-2024',
       '30-09-2024', '31-10-2024', '30-11-2024', '31-12-2024', '31-01-2025',
       '28-02-2025', '31-03-2025', '30-04-2025', '31-05-2025', '30-06-2025',
       '31-07-2025', '31-08-2025', '30-09-2025', '31-10-2025', '30-11-2025',
       '31-12-2025'],
      dtype='object')

## Base Demanda de Produtos:

In [81]:
try:
    engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")
    with engine.connect() as conn:
        # Objeto que vai armazenar consultas:
        script = {}
        for i in tqdm(range(0,periods,1)):
            if i>0:
                sleep(60)
    
            script[i] = \
            f"""
            -- DEMANDA DE PRODUTO POR LOJA!
                WITH CALENDARIO AS (
                    SELECT 
                        TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') + LEVEL - 1 AS data
                    FROM dual
                    CONNECT BY LEVEL <= TO_DATE('{range_date_end[i]}', 'DD-MM-YYYY') - TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') + 1
                order by 1 asc
                )

                -- CTE para garantir todas as combinações de data, produto e loja
                , BASE AS (
                    SELECT 
                        C.DATA,
                        Z.CODIGO,
                        Z.COD_EMPRESA
                        
                    FROM CALENDARIO C
                    CROSS JOIN (
                        SELECT DISTINCT P.CODIGO, B.COD_EMPRESA
                        FROM PRODB B
                        JOIN PROD P ON P.CODIGO =B.CODIGO
                        
                        WHERE P.ATIVO = 'S'
                        AND P.COMPRADOR = '6219' 
                        AND NVL(P.LG_SERVICO, 'N') = 'N'
                        AND P.LINHA IN ('UAA', 'UAB', 'UAC', 'UAD', 'UAG', 'UAH', 'UAI', 'UAK', 'UBI', 'UBJ', 'UDB', 'UFA', 'UFC')
                    ) Z
                )


                , PADRAO_COMPRA AS (
                    SELECT DISTINCT M.CODIGO, M.COD_EMPRESA, NVL(M.PADRAO_COMPRA_COD_EMPRESA, M.COD_EMPRESA) AS PADRAO_COMPRA
                    FROM PROD_MIX M
                    WHERE M.ID_LOJA = 0
                )


                SELECT B.DATA, B.COD_EMPRESA, B.CODIGO,  
                    V.UND_VENDA, NVL(V.QTD_VENDIDA,0) as QTD_VENDA, 
                    M.PADRAO_COMPRA
                    , PRC_VENDA
                    
                FROM BASE B
                LEFT JOIN PADRAO_COMPRA M ON B.COD_EMPRESA = M.COD_EMPRESA AND B.CODIGO = M.CODIGO
                LEFT JOIN (
                SELECT NVL(R.DATA_PEDIDO, S.DATAEMISSAO) AS DATA,
                    P.CODIGO,
                    S.COD_EMPRESA,
                    S.UNVENDA3 AS UND_VENDA,
                    SUM(S.QTSV * NVL(S.CONVERSAO,1)) AS QTD_VENDIDA
                    , MEDIAN(ROUND((S.PRCTOTAL - NVL(S.VALDESC,0)) / (S.QTSV * S.CONVERSAO) , 2)) AS PRC_VENDA -- ATT AJUSTAR DEPOIS PARA MÉDIA PONDERADA
                    -- ATT ACRESCENTAR PREÇO ONDE NÃO HOUVE VENDA 
                    
                    
                FROM SAIDM_IT S
                JOIN SAIDM SM
                ON SM.NUMNOTA = S.NUMNOTA
                AND S.COD_EMPRESA = SM.COD_EMPRESA
                JOIN PROD P
                ON SUBSTR(S.PRODUTO,1,LENGTH(S.PRODUTO)-2) = P.CODIGO



                LEFT JOIN SAID_IT_RENTA R
                ON R.COD_EMPRESA = S.COD_EMPRESA
                AND R.NUMNOTA = S.NUMNOTA
                AND R.PRODUTO = S.PRODUTO
                AND R.DATAEMISSAO = S.DATAEMISSAO
                AND R.TIPO_CUSTO = 'P'

                WHERE P.ATIVO = 'S'
                AND P.COMPRADOR = '6219' 
                AND NVL(P.LG_SERVICO, 'N') = 'N'
                AND P.LINHA IN ('UAA', 'UAB', 'UAC', 'UAD', 'UAG', 'UAH', 'UAI', 'UAK', 'UBI', 'UBJ', 'UDB', 'UFA', 'UFC')
                AND P.ATIVO ='S'
                AND S.TIPOSAIDA = '1'
                AND ((NVL(SM.STATUS, '0') != 'C' AND NVL(SM.STATUS, '0')  != 'D') OR
                    (NVL(SM.STATUS, '0')  = 'D' AND NVL(SM.CODENTREG, 0) != 101))

                    
                AND S.DATAEMISSAO BETWEEN TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') AND TO_DATE('{range_date_end[i]}', 'DD-MM-YYYY')
                
                GROUP BY NVL(R.DATA_PEDIDO, S.DATAEMISSAO),
                    P.CODIGO,
                    S.COD_EMPRESA,
                    S.UNVENDA3
                    
                ORDER BY COD_EMPRESA, CODIGO, DATA 
                ) V ON B.DATA = V.DATA AND B.COD_EMPRESA = V.COD_EMPRESA AND B.CODIGO = V.CODIGO
            """
            
            df_ = pd.read_sql(script[i], con = conn)
        
            # Verifica se é a primeira iteração. Caso for, inicializa um DF vazio
            if i == 0:
                df = df_
            else:
                # Empilha os resultados de cada mês
                df = pd.concat([df, df_], ignore_index = True, axis = 0)
        
        engine.dispose()
    
except Exception as e:
    print(f'Erro: {e}')
    
else:
    print("Dados Coletados")
    
finally:
    # Desconectando conexão com o Oracle
    conn.close()
    print('Conxeão Fechada')

100%|██████████| 4/4 [04:57<00:00, 74.27s/it]

Dados Coletados
Conxeão Fechada


In [82]:
# Execução 1: Janeiro a Abril de 2025
df_2025_1 = df.copy()
df_2025_1.to_csv('./Data/Demanda/df_2025_1.csv')
print(f'2025 salvo: {len(df_2025_1)} registros')
df_2025_1.describe()

2025 salvo: 1012800 registros


,data,cod_empresa,qtd_venda,padrao_compra,prc_venda
count,1012800,1.012800e+06,1.012800e+06,937080.000000,46993.000000
mean,2025-03-01 12:00:00.000000768,4.049929e+01,1.327518e-01,59.075170,147.479505
min,2025-01-01 00:00:00,1.000000e+00,0.000000e+00,1.000000,5.100000
25%,2025-01-30 18:00:00,4.000000e+00,0.000000e+00,8.000000,41.900000
50%,2025-03-01 12:00:00,8.000000e+00,0.000000e+00,80.000000,89.900000
75%,2025-03-31 06:00:00,8.200000e+01,0.000000e+00,82.000000,169.000000
max,2025-04-30 00:00:00,9.400000e+01,1.500000e+02,94.000000,6359.900000
std,NaN,4.061012e+01,1.053741e+00,36.893971,196.313910


In [ ]:
# Execução 2: Maio a Agosto de 2025
df_2025_2 = df.copy()
df_2025_2.to_csv('./Data/Demanda/df_2025_2.csv')
print(f'2025_2 salvo: {len(df_2025_2)} registros')
df_2025_2.describe()

2025_2 salvo: 1038120 registros


,data,cod_empresa,qtd_venda,padrao_compra,prc_venda
count,1038120,1.038120e+06,1.038120e+06,960507.00000,53465.000000
mean,2025-06-30 23:59:59.999999744,4.049929e+01,1.960727e-01,59.07517,158.837587
min,2025-05-01 00:00:00,1.000000e+00,0.000000e+00,1.00000,6.360000
25%,2025-05-31 00:00:00,4.000000e+00,0.000000e+00,8.00000,41.900000
50%,2025-07-01 00:00:00,8.000000e+00,0.000000e+00,80.00000,92.900000
75%,2025-08-01 00:00:00,8.200000e+01,0.000000e+00,82.00000,178.900000
max,2025-08-31 00:00:00,9.400000e+01,2.380000e+02,94.00000,5659.900000
std,NaN,4.061012e+01,1.707009e+00,36.89397,222.291145


In [ ]:
# Execução 3: Setembro a Dezembro de 2025
df_2025_3 = df.copy()
df_2025_3.to_csv('./Data/Demanda/df_2025_3.csv')
print(f'2025_3 salvo: {len(df_2025_2)} registros')

2025_3 salvo: 1029680 registros


In [74]:
df_2025_3.describe()

,data,cod_empresa,qtd_venda,padrao_compra,prc_venda
count,1029680,1.029680e+06,1.029680e+06,952698.000000,51416.000000
mean,2025-10-31 12:00:00,4.049929e+01,1.574586e-01,59.075170,159.545629
min,2025-09-01 00:00:00,1.000000e+00,0.000000e+00,1.000000,6.690000
25%,2025-10-01 00:00:00,4.000000e+00,0.000000e+00,8.000000,44.900000
50%,2025-10-31 12:00:00,8.000000e+00,0.000000e+00,80.000000,92.900000
75%,2025-12-01 00:00:00,8.200000e+01,0.000000e+00,82.000000,182.900000
max,2025-12-31 00:00:00,9.400000e+01,1.260000e+02,94.000000,5389.900000
std,NaN,4.061012e+01,1.341732e+00,36.893971,209.620032


In [ ]:
# Execução 4: Setembro a Dezembro de 2024
df_2024_3 = df.copy()
df_2024_3.to_csv('./Data/Demanda/df_2024_3.csv')
print(f'2024_3 salvo: {len(df_2024_3)} registros')
df_2024_3.describe()

2024_3 salvo: 1029680 registros


,data,cod_empresa,qtd_venda,padrao_compra,prc_venda
count,1029680,1.029680e+06,1.029680e+06,952698.000000,48877.000000
mean,2024-10-31 12:00:00,4.049929e+01,1.376039e-01,59.075170,151.712771
min,2024-09-01 00:00:00,1.000000e+00,0.000000e+00,1.000000,5.535000
25%,2024-10-01 00:00:00,4.000000e+00,0.000000e+00,8.000000,42.900000
50%,2024-10-31 12:00:00,8.000000e+00,0.000000e+00,80.000000,89.900000
75%,2024-12-01 00:00:00,8.200000e+01,0.000000e+00,82.000000,169.900000
max,2024-12-31 00:00:00,9.400000e+01,2.050000e+02,94.000000,6359.900000
std,NaN,4.061012e+01,1.182522e+00,36.893971,208.502977


In [ ]:
# Execução 5: Maio a Agosto de 2024
df_2024_2 = df.copy()
df_2024_2.to_csv('./Data/Demanda/df_2024_2.csv')
print(f'2024_2 salvo: {len(df_2024_2)} registros')
df_2024_2.describe()

2024_2 salvo: 1038120 registros


,data,cod_empresa,qtd_venda,padrao_compra,prc_venda
count,1038120,1.038120e+06,1.038120e+06,960507.00000,61911.000000
mean,2024-07-01 00:00:00.000000512,4.049929e+01,1.850865e-01,59.07517,143.184104
min,2024-05-01 00:00:00,1.000000e+00,0.000000e+00,1.00000,4.800000
25%,2024-05-31 00:00:00,4.000000e+00,0.000000e+00,8.00000,41.900000
50%,2024-07-01 00:00:00,8.000000e+00,0.000000e+00,80.00000,87.400000
75%,2024-08-01 00:00:00,8.200000e+01,0.000000e+00,82.00000,159.900000
max,2024-08-31 00:00:00,9.400000e+01,8.500000e+01,94.00000,5445.390000
std,NaN,4.061012e+01,1.349921e+00,36.89397,198.999640


In [61]:
# Execução 6: Janeiro a Abril de 2024
df_2024_1 = df.copy()
df_2024_1.to_csv('./Data/Demanda/df_2024_1.csv')
print(f'2024_1 salvo: {len(df_2024_1)} registros')
df_2024_1.describe()

2024_1 salvo: 1021240 registros


,data,cod_empresa,qtd_venda,padrao_compra,prc_venda
count,1021240,1.021240e+06,1.021240e+06,944889.000000,49424.000000
mean,2024-03-01 00:00:00.000000512,4.049929e+01,1.336924e-01,59.075170,140.797631
min,2024-01-01 00:00:00,1.000000e+00,0.000000e+00,1.000000,5.515000
25%,2024-01-31 00:00:00,4.000000e+00,0.000000e+00,8.000000,43.900000
50%,2024-03-01 00:00:00,8.000000e+00,0.000000e+00,80.000000,89.000000
75%,2024-03-31 00:00:00,8.200000e+01,0.000000e+00,82.000000,159.000000
max,2024-04-30 00:00:00,9.400000e+01,2.600000e+02,94.000000,4299.000000
std,NaN,4.061012e+01,1.084712e+00,36.893971,195.077209


In [64]:
# Execução 7: Janeiro a Abril de 2023
df_2023_1 = df.copy()
df_2023_1.to_csv('./Data/Demanda/df_2023_1.csv')
print(f'2023_1 salvo: {len(df_2023_1)} registros')
df_2023_1.describe()

2023_1 salvo: 1012800 registros


,data,cod_empresa,qtd_venda,padrao_compra,prc_venda
count,1012800,1.012800e+06,1.012800e+06,937080.000000,46398.000000
mean,2023-03-01 12:00:00.000000256,4.049929e+01,1.267022e-01,59.075170,126.537096
min,2023-01-01 00:00:00,1.000000e+00,0.000000e+00,1.000000,5.030000
25%,2023-01-30 18:00:00,4.000000e+00,0.000000e+00,8.000000,39.800000
50%,2023-03-01 12:00:00,8.000000e+00,0.000000e+00,80.000000,79.900000
75%,2023-03-31 06:00:00,8.200000e+01,0.000000e+00,82.000000,156.000000
max,2023-04-30 00:00:00,9.400000e+01,1.460000e+02,94.000000,3999.900000
std,NaN,4.061012e+01,1.102050e+00,36.893971,156.686266


In [67]:
# Execução 8: Maio a Agosto de 2023
df_2023_2 = df.copy()
df_2023_2.to_csv('./Data/Demanda/df_2023_2.csv')
print(f'2023_2 salvo: {len(df_2023_2)} registros')
df_2023_2.describe()

2023_2 salvo: 1038120 registros


,data,cod_empresa,qtd_venda,padrao_compra,prc_venda
count,1038120,1.038120e+06,1.038120e+06,960507.00000,56318.000000
mean,2023-07-01 00:00:00.000000768,4.049929e+01,1.860604e-01,59.07517,135.485357
min,2023-05-01 00:00:00,1.000000e+00,0.000000e+00,1.00000,4.900000
25%,2023-05-31 00:00:00,4.000000e+00,0.000000e+00,8.00000,40.900000
50%,2023-07-01 00:00:00,8.000000e+00,0.000000e+00,80.00000,83.000000
75%,2023-08-01 00:00:00,8.200000e+01,0.000000e+00,82.00000,159.000000
max,2023-08-31 00:00:00,9.400000e+01,2.070000e+02,94.00000,4999.900000
std,NaN,4.061012e+01,1.639159e+00,36.89397,182.511278


In [73]:
# Execução 9: Setembro a Dezembro de 2023
df_2023_3 = df.copy()
df_2023_3.to_csv('./Data/Demanda/df_2023_3.csv')
print(f'2023_3 salvo: {len(df_2023_3)} registros')
df_2023_3.describe()

2023_3 salvo: 1029680 registros


,data,cod_empresa,qtd_venda,padrao_compra,prc_venda
count,1029680,1.029680e+06,1.029680e+06,952698.000000,51995.000000
mean,2023-10-31 12:00:00.000000256,4.049929e+01,1.634848e-01,59.075170,135.867307
min,2023-09-01 00:00:00,1.000000e+00,0.000000e+00,1.000000,5.575000
25%,2023-10-01 00:00:00,4.000000e+00,0.000000e+00,8.000000,39.900000
50%,2023-10-31 12:00:00,8.000000e+00,0.000000e+00,80.000000,79.900000
75%,2023-12-01 00:00:00,8.200000e+01,0.000000e+00,82.000000,149.900000
max,2023-12-31 00:00:00,9.400000e+01,2.010000e+02,94.000000,5299.000000
std,NaN,4.061012e+01,1.557881e+00,36.893971,195.335751


In [83]:
df_final = pd.concat([df_2025_1, df_2025_2, df_2025_3, df_2024_1, df_2024_2, df_2024_3, df_2023_1, df_2023_2, df_2023_3], ignore_index=True)

In [43]:
# for i in tqdm(range(0,periods,1)):
#     df.\
#     loc[(df['data'] >= range_date_start[i]) & (df['data'] <= range_date_end[i])].\
#     to_excel(f'./Data/Demanda/demanda_{range_date_start[i]}.xlsx',  index=False)

## Base Estoque dia-dia

In [9]:
try:
    engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")
    with engine.connect() as conn:
        # Objeto que vai armazenar consultas:
        script = {}
        for i in tqdm(range(0,periods,1)):
            if i>0:
                sleep(60)
    
            script[i] = \
            f"""
             -- ESTOQUE DIARIO
            WITH CALENDARIO AS (
                SELECT 
                    TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') + LEVEL - 1 AS data
                FROM dual
                CONNECT BY LEVEL <= TO_DATE('{range_date_end[i]}', 'DD-MM-YYYY') - TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') + 1
            order by 1 asc
            )

            -- CTE para garantir todas as combinações de data, produto e loja
            , BASE AS (
                SELECT 
                    C.DATA,
                    Z.CODIGO,
                    Z.COD_EMPRESA
                                    
                FROM CALENDARIO C
                CROSS JOIN (
                    SELECT DISTINCT P.CODIGO, B.COD_EMPRESA
                    FROM PRODB B
                    JOIN PROD P ON P.CODIGO =B.CODIGO
                                    
                    WHERE P.ATIVO = 'S'
                    AND P.COMPRADOR = '6219' 
                    AND NVL(P.LG_SERVICO, 'N') = 'N'
                    AND P.LINHA IN ('UAA', 'UAB', 'UAC', 'UAD', 'UAG', 'UAH', 'UAI', 'UAK', 'UBI', 'UBJ', 'UDB', 'UFA', 'UFC')
                ) Z
            )

            SELECT B.DATA, 
                B.COD_EMPRESA, 
                B.CODIGO,  
                E.ESTOQUE_DIARIO

                
            FROM BASE B
            LEFT JOIN (
                        SELECT B.DATA,
                            B.COD_EMPRESA,
                            B.CODIGO,
                            (NVL(qtde1, 0)+   NVL(qtde2, 0)+   NVL(qtde3, 0)+  NVL(qtde4, 0)+  NVL(qtde5, 0) +
                            NVL(qtde6, 0)+   NVL(qtde7, 0)+   NVL(qtde8, 0)+  NVL(qtde9, 0)+  NVL(qtde10, 0)+
                            NVL(qtde11, 0)+  NVL(qtde12, 0)+  NVL(qtde13, 0)+ NVL(qtde14, 0)+ NVL(qtde15, 0)+
                            NVL(qtde16, 0)+  NVL(qtde17, 0)+  NVL(qtde18, 0)+ NVL(qtde19, 0) + NVL(QTDE20,0)) ESTOQUE_DIARIO
                        FROM PRODB_DIARIO_{range_date[i]} B
                        JOIN PROD P ON P.CODIGO = B.CODIGO 
                        WHERE P.ATIVO = 'S'
                            AND P.COMPRADOR = '6219' 
                            AND NVL(P.LG_SERVICO, 'N') = 'N'
                            AND P.LINHA IN ('UAA', 'UAB', 'UAC', 'UAD', 'UAG', 'UAH', 'UAI', 'UAK', 'UBI', 'UBJ', 'UDB', 'UFA', 'UFC')


            ) E ON B.DATA = E.DATA AND B.COD_EMPRESA = E.COD_EMPRESA AND B.CODIGO = E.CODIGO
            

            """
            
            df_estoque_mes = pd.read_sql(script[i], con = conn)
        
            # Verifica se é a primeira iteração. Caso for, inicializa um DF vazio
            if i == 0:
                df_est = df_estoque_mes
            else:
                # Empilha os resultados de cada mês
                df_est = pd.concat([df_est, df_estoque_mes], ignore_index = True, axis = 0)
            
        engine.dispose()
            
except Exception as e:
    print(f'Erro: {e}')
    
else:
    print("Dados Coletados")
    
finally:
    # Desconectando conexão com o Oracle

    conn.close()
    print('Conxeão Fechada')

100%|██████████| 36/36 [2:22:09<00:00, 236.93s/it]   

Dados Coletados
Conxeão Fechada


In [10]:
try:
    engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")
    with engine.connect() as conn:
        # Objeto que vai armazenar consultas:
        script = {}
        for i in tqdm(range(0,periods,1)):
            if i>0:
                sleep(60)
    
            script[i] = \
            f"""
              SELECT F.DT_REFERENCIA as data,
                            DF.CD_EMPRESA as cod_empresa,
                            DP.CD_PRODUTO AS CODIGO, 
                            F.ESTOQUE AS ESTOQUE_DIARIO
                        
                        FROM FATO_COMESTQ_{range_date[i]} F
                        JOIN DIM_PRODUTO DP ON F.DIM_PRODUTO = DP.DIM_PRODUTO
                        JOIN DIM_FILIAL DF ON DF.DIM_FILIAL = F.DIM_FILIAL
                        JOIN PROD P ON P.CODIGO = DP.CD_PRODUTO
                        
                        WHERE 1=1
                            AND DF.CD_EMPRESA <> '999' -- ATT Removemos ecommerce porque no modelo piloto não separamos ecommerce de loja  
                            AND P.ATIVO = 'S'
                            AND P.COMPRADOR = '6219' 
                            AND NVL(P.LG_SERVICO, 'N') = 'N'
                            AND P.LINHA IN ('UAA', 'UAB', 'UAC', 'UAD', 'UAG', 'UAH', 'UAI', 'UAK', 'UBI', 'UBJ', 'UDB', 'UFA', 'UFC')
            """
            
            est_fechamento = pd.read_sql(script[i], con = conn)
        
            # Verifica se é a primeira iteração. Caso for, inicializa um DF vazio
            if i == 0:
                df_est_fechamento = est_fechamento
            else:
                # Empilha os resultados de cada mês
                df_est_fechamento = pd.concat([df_est_fechamento, est_fechamento], ignore_index = True, axis = 0)
            
        engine.dispose()
            
except Exception as e:
    print(f'Erro: {e}')
    
else:
    print("Dados Coletados")
    
finally:
    # Desconectando conexão com o Oracle

    conn.close()
    print('Conxeão Fechada')

100%|██████████| 36/36 [46:00<00:00, 76.67s/it] 

Dados Coletados
Conxeão Fechada


In [11]:
df_est = df_est.merge(df_est_fechamento, how='left', on = ['data', 'cod_empresa', 'codigo'], validate = '1:1')

In [12]:
df_est['estoque_diario'] = np.where(df_est['estoque_diario_x'].notna(), df_est['estoque_diario_x'],df_est['estoque_diario_y'])
df_est = df_est.drop(['estoque_diario_x', 'estoque_diario_y'], axis = 1)

In [30]:
df.value_counts()

data        cod_empresa  codigo  und_venda  qtd_venda  padrao_compra  prc_venda
2026-01-01  4            463844  UN         1          4.0            108.9        36
2026-01-05  2            358186  UN         1          80.0           780.9        36
            3            495428  UN         3          80.0           349.9        36
                         495427  UN         1          80.0           349.9        36
                         480707  UN         1          80.0           499.9        36
                                                                                   ..
2026-01-03  4            476364  UN         2          4.0            89.9         36
                         473320  UN         1          4.0            159.9        36
                         468620  UN         2          80.0           56.9         36
                         468618  UN         1          80.0           103.9        36
2026-01-06  9            83166   UN         3          9.0  

In [84]:
df_final.value_counts()

data        cod_empresa  codigo  und_venda  qtd_venda  padrao_compra  prc_venda
2023-01-02  1            102242  UN         2          1.0            34.9         1
2024-12-21  2            369861  UN         1          2.0            99.0         1
            3            117790  UN         2          80.0           157.9        1
                         117592  UN         1          80.0           35.9         1
            2            83166   UN         5          2.0            14.9         1
                                                                                  ..
2024-01-03  7            210848  UN         1          80.0           50.9         1
                         207083  UN         4          80.0           109.9        1
                         189579  UN         1          80.0           56.9         1
                         168593  UN         3          80.0           359.0        1
2025-12-31  9            68809   UN         1          9.0            

In [29]:
df_est.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9250240 entries, 0 to 9250239
Data columns (total 4 columns):
 #   Column          Dtype         
---  ------          -----         
 0   data            datetime64[ns]
 1   cod_empresa     int64         
 2   codigo          object        
 3   estoque_diario  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 282.3+ MB


In [85]:
base_demanda = df_final.merge(df_est, how='left', on = ['data', 'cod_empresa', 'codigo'], validate = '1:1')
base_demanda.head()

,data,cod_empresa,codigo,und_venda,qtd_venda,padrao_compra,prc_venda,estoque_diario
0,2025-01-11,1,117592,UN,1,80.0,35.90,7.0
1,2025-01-18,1,117592,UN,1,80.0,35.90,6.0
2,2025-01-31,1,117779,UN,1,80.0,92.90,11.0
3,2025-01-18,1,117790,UN,1,80.0,154.90,13.0
4,2025-01-25,1,117790,UN,2,80.0,147.16,11.0


In [86]:
base_demanda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9250240 entries, 0 to 9250239
Data columns (total 8 columns):
 #   Column          Dtype         
---  ------          -----         
 0   data            datetime64[ns]
 1   cod_empresa     int64         
 2   codigo          object        
 3   und_venda       object        
 4   qtd_venda       int64         
 5   padrao_compra   float64       
 6   prc_venda       float64       
 7   estoque_diario  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 564.6+ MB


In [90]:
base_demanda.describe()

,data,cod_empresa,qtd_venda,padrao_compra,prc_venda,estoque_diario
count,9250240,9.250240e+06,9.250240e+06,8.558664e+06,466797.000000,6.254863e+06
mean,2024-07-01 12:00:00.000001792,4.049929e+01,1.578668e-01,5.907517e+01,144.453425,2.224124e+01
min,2023-01-01 00:00:00,1.000000e+00,0.000000e+00,1.000000e+00,4.800000,0.000000e+00
25%,2023-10-01 18:00:00,4.000000e+00,0.000000e+00,8.000000e+00,41.900000,0.000000e+00
50%,2024-07-01 12:00:00,8.000000e+00,0.000000e+00,8.000000e+01,87.900000,0.000000e+00
75%,2025-04-01 06:00:00,8.200000e+01,0.000000e+00,8.200000e+01,169.000000,6.000000e+00
max,2025-12-31 00:00:00,9.400000e+01,2.600000e+02,9.400000e+01,6359.900000,9.207000e+03
std,NaN,4.061011e+01,1.358063e+00,3.689395e+01,197.554782,1.231937e+02


In [49]:
del df_est_fechamento

In [89]:
for i in tqdm(range(0,periods,1)):
    base_demanda.\
    loc[(base_demanda['data'] >= range_date_start[i]) & (base_demanda['data'] <= range_date_end[i])].\
    to_csv(f'./Data/Demanda/demanda_{range_date_start[i]}',  index=False, encoding = 'utf-8')

100%|██████████| 36/36 [03:43<00:00,  6.22s/it]


## Pedidos Fornecedor

In [ ]:
# try:
#     engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")
#     with engine.connect() as conn:
#         # Objeto que vai armazenar consultas:
#         script = {}
#         for i in tqdm(range(0,periods,1)):
#             if i>0:
#                 sleep(60)
    
#             script[i] = \
#               f"""
#                    -- ESTOQUE_COMPRAS
#                     WITH ESCOPO AS (
#                     SELECT DISTINCT P.CODIGO, B.COD_EMPRESA
#                             FROM PRODB B
#                             JOIN PROD P ON P.CODIGO =B.CODIGO
                                                    
#                             WHERE P.ATIVO = 'S'
#                             AND P.COMPRADOR = '6219' 
#                             AND NVL(P.LG_SERVICO, 'N') = 'N'
#                             AND P.LINHA IN ('UAA', 'UAB', 'UAC', 'UAD', 'UAG', 'UAH', 'UAI', 'UAK', 'UBI', 'UBJ', 'UDB', 'UFA', 'UFC')
#                     )
                                
#                     -- CNPJ  SKU Filial de Entrega Tipo_Destino  Lead_Time_Dias  Ciclo_Pedido_Dias Faturamento Mínimo  Multiplo_Palete Multiplo_Carreta  Custo_Unitario  Custo_Frete_Palete  Estoque_Disponivel  Estoque_Transito

#                     SELECT 
#                         NVL(E2.DATAENTRADA, E.DATAENTRADA) AS DATA_ENTRADA,
#                         E.NUMPEDIDO,
#                         P.FORNECEDOR AS CNPJ,
#                         EI.PRODUTO,
#                         DECODE(NVL(E.CGC_TRANSF_AUTO, ' '), ' ', E.COD_EMPRESA, PA.COD_EMPRESA) AS FILIAL_ENTREGA, 
#                         CASE WHEN DECODE(NVL(E.CGC_TRANSF_AUTO, ' '), ' ', E.COD_EMPRESA, PA.COD_EMPRESA) >= 80
#                             THEN 'CD'
#                             ELSE 'LOJA'
#                             END AS TIPO_DESTINO,
#                         (NVL(E2.DATAENTRADA, E.DATAENTRADA) - E.DATAPEDIDO) AS LEAD_TIME_DIAS,
#                         NULL AS CICLO_PEDIDO,
#                         F.FAT_MINIMO AS FATURAMENTO_MINIMO,    
#                         EI.PRECO_UNIT AS PRECO_UNIT,
#                         EI.QUANTIDADE AS QTD_ENTREGUE,
#                         EI.CONVERSAO AS CONVERSAO_UN_ESTOQUE,
#                         NULL AS ESTOQUE_DISPONIVEL,
#                         EI.QUANTIDADE * NVL(DECODE(EI.CONVERSAO,0,1,EI.CONVERSAO), 1) AS QTD_CONVERTIDA
                                    
#                     FROM ENTR_IT EI

#                     JOIN ESCOPO ES ON ES.CODIGO = EI.PRODUTO AND ES.COD_EMPRESA = EI.COD_EMPRESA

#                     JOIN ENTR E 
#                     ON E.CGC_EMITENTE = EI.CGC_EMITENTE 
#                     AND E.NUMNOTA      = EI.NUM_NOTA
#                     AND E.COD_EMPRESA  = EI.COD_EMPRESA
#                     AND E.SERIE        = EI.SERIE 

#                     LEFT JOIN (
#                         SELECT DISTINCT P26.COD_EMPRESA, P26.CGC
#                         FROM PARAM026 P26
                        
#                     ) PA ON PA.CGC = E.CGC_TRANSF_AUTO

#                     JOIN PROD P ON P.CODIGO = EI.PRODUTO
#                     JOIN FORN F ON F.CGC = P.FORNECEDOR
                    
#                     LEFT JOIN (
#                         SELECT DISTINCT V.CHAVE_NFE_ORIGEM_TRANSF, V.CHAVE_NFE, V.COD_EMPRESA
#                         FROM VCEGA V
#                         WHERE 1=1
#                         AND NVL(V.STATUS,' ') = 'B'
#                         AND V.TIPO = '11'
#                         AND NVL(V.CHAVE_NFE_ORIGEM_TRANSF, ' ') <> ' '
                    
#                     ) V ON E.CHAVE_NFE = V.CHAVE_NFE_ORIGEM_TRANSF    -- AQUI EU PEGO SE A ENTRADA SE REFERE A ORIGEM DE UMA TRANSFERÊNCIA AUTOMÁTICA. 

#                     LEFT JOIN ENTR E2 ON V.CHAVE_NFE = E2.CHAVE_NFE AND E2.TIPO = '11'   -- USO ESSE JOIN PRA SABER A DATA ENTRADA QUE A MERCADORIA DEU NO DESTINO FINAL.
                    
#                     WHERE 1=1
#                     AND NVL(E2.DATAENTRADA, E.DATAENTRADA) BETWEEN TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') AND TO_DATE('{range_date_end[i]}', 'DD-MM-YYYY')
#                     AND E.TIPO = '1'
#                     AND NVL(E.STATUS, ' ') <> 'C'
#                   """
            
#             ped_fornecedor = pd.read_sql(script[i], con = conn)

#             ped_fornecedor.to_csv(f'./Data/pedidos_fornecedor/pedidos_fornecedor_{range_date_start[i]}.csv',  index=False, encoding = 'utf-8')
            
    
# except Exception as e:
#     print(f'Erro: {e}')
    
# else:
#     print("Dados Coletados")
    
# finally:
#     # Desconectando conexão com o Oracle
#     conn.close()
#     print('Conxeão Fechada')

  0%|          | 0/36 [00:00<?, ?it/s]

## Transferencias CD

In [ ]:
# try:
#     engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")
#     with engine.connect() as conn:
#         # Objeto que vai armazenar consultas:
#         script = {}

#         script = \
#             f"""
#             SELECT S.MOTIVO, 
#                     S.NU_PEDIDO,
#                     S.DT_MOVIMENTO,
#                     S.COD_EMPRESA_ORIGEM,
#                     S.COD_EMPRESA,
#                     P.CODIGO,
#                     S.DT_MOVIMENTO  - S.DT_SOLI AS LEAD_TIME_DIAS,
#                     S.QTD_ATENDIDA,
#                     B.CUE AS CUSTO_UNIT
                                            
#                 FROM SOLI S
#                 JOIN PROD P ON P.CODIGO = SUBSTR(S.COD_PRODUTO,1, LENGTH(S.COD_PRODUTO)-2)
#                 JOIN PRODB B ON B.CODIGO = P.CODIGO AND B.COD_EMPRESA = S.COD_EMPRESA

#                 WHERE NVL(S.MOTIVO, ' ') NOT LIKE ('TRANSF%AUTO%')
#                 AND NVL(S.STATUS, ' ') NOT IN ('E')
#                 AND S.COD_EMPRESA_ORIGEM >= 80
#                 AND P.ATIVO = 'S'
#                 AND P.COMPRADOR = '6219' 
#                 AND NVL(P.LG_SERVICO, 'N') = 'N'
#                 AND P.LINHA IN ('UAA', 'UAB', 'UAC', 'UAD', 'UAG', 'UAH', 'UAI', 'UAK', 'UBI', 'UBJ', 'UDB', 'UFA', 'UFC')
#                 AND NVL(S.MOTIVO_CANC, 0) = 0
#                 AND S.QUANTIDADE >0  

#                 ORDER BY 1 ASC
  
#             """
        
#         transferencias = pd.read_sql(script, con = conn)    
    
# except Exception as e:
#     print(f'Erro: {e}')
    
# else:
#     print("Dados Coletados")
    
# finally:
#     # Desconectando conexão com o Oracle
    
#     conn.close()
#     print('Conxeão Fechada')

In [ ]:
# transferencias = transferencias.merge(df_est, how='left', on = ['data', 'cod_empresa', 'codigo'], validate = '1:1')

## Campanhas

In [ ]:
from sqlalchemy import create_engine,text, types
import pandas as pd
from pandas import read_sql
from datetime import datetime,timedelta
from getpass import getpass
from dateutil.relativedelta import relativedelta
import gc

In [ ]:
def get_sqlalchemy_dtypes(df):
    type_mapping = {
        "object": types.String,
        "int64": types.Integer,
        "float64": types.Float,
        "datetime64[ns]": types.DateTime,
        "datetime64[ns, UTC]": types.DateTime,
        "bool": types.Boolean
    }
    dtypes = {}
    for col, dtype in df.dtypes.items():
        sql_type = type_mapping.get(str(dtype), types.String)
        dtypes[col] = sql_type
    return dtypes

engine_bifc = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")


In [ ]:
#Campanhas

query_linha = f""" 
SELECT DISTINCT 
          F.COD_EMPRESA,
          F.COD_CAMPANHA_FILIAL AS CODIGO_CAMPANHA,
          F.NM_CAMPANHA_FILIAL AS NOME_CAMPANHA,
          F.DT_LOJA_INICIO AS START_DATE,
          F.DT_LOJA_FINAL AS END_DATE,
          M.NM_CAMPANHA AS CAMPANHA_MASTER_ASSOCIADA,
          T.NM_TIPO_CAMPANHA

FROM SFC_CAMP_FILIAL F
LEFT JOIN SFC_CAMP_MASTER M ON F.COD_CAMPANHA_MASTER = M.COD_CAMPANHA_MASTER
LEFT JOIN SFC_CAMP_TIPO T ON M.COD_TIPO_CAMPANHA = T.COD_TIPO_CAMPANHA
WHERE F.DT_LOJA_INICIO > ADD_MONTHS(SYSDATE, -36)
  AND F.DT_LOJA_FINAL <= '31/12/2026'

ORDER BY START_DATE ASC


    """

try:
    df_camp = pd.read_sql(text(query_linha), engine_bifc)
    df_camp.to_csv(r"./Data/campanhas/campanhas.csv", index=False, encoding = 'utf-8')
    del df_camp        
except Exception as e:
    print(f"❌ Erro ao acessar {query_linha}: {e}")


    
engine_bifc.dispose()

In [ ]:
#Produtos_locais_campanhas

query_linha = f""" 

SELECT CAST(CP.COD_CAMPANHA_FILIAL AS VARCHAR(20)) AS CODIGO_CAMPANHA,
       P.CODIGO AS CODIGO_PROD,
       CAST(CP.COD_EMPRESA AS VARCHAR(4)) AS COD_EMPRESA,
       ROUND(CP.PRVD1_CAMPANHA * ( 1 -CP.DESCONTO/100), 2) AS PRECO_CAMPANHA

FROM SFC_CAMP_PROD CP
JOIN PROD P ON P.CODIGO = CP.COD_PRODUTO

WHERE EXISTS ( 
      SELECT 1
        FROM SFC_CAMP_FILIAL F
        LEFT JOIN SFC_CAMP_MASTER M ON F.COD_CAMPANHA_MASTER = M.COD_CAMPANHA_MASTER
        LEFT JOIN SFC_CAMP_TIPO T ON M.COD_TIPO_CAMPANHA = T.COD_TIPO_CAMPANHA
        WHERE F.DT_LOJA_INICIO > ADD_MONTHS(SYSDATE, -36)
          AND F.DT_LOJA_FINAL <= SYSDATE
          AND F.COD_EMPRESA = CP.COD_EMPRESA
          AND F.COD_CAMPANHA_FILIAL = CP.COD_CAMPANHA_FILIAL
)
AND CP.PRVD1_CAMPANHA > 0

    """

try:
    camp_prod = pd.read_sql(text(query_linha), engine_bifc)
    camp_prod.to_csv(r"./Data/campanhas/produtos_campanha.csv", index=False, encoding = 'utf-8')
    del camp_prod             
except Exception as e:
    print(f"❌ Erro ao acessar {query_linha}: {e}")


    
engine_bifc.dispose()